In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import itertools
import tensorflow as tf
from keras.models import load_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
with open('done_preprocessing.csv', 'r') as f:
    data = pd.read_csv(f)
data.head()

In [ ]:

# Selecting features and target
X = data.drop(columns=['power', 'time'])
y = data['power']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Applying MinMaxScaler
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# Scaling the features
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Scaling the target variable
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

print(X_train_scaled.shape, y_train_scaled.shape, X_test_scaled.shape, y_test_scaled.shape)

# Define hyperparameter grid
config = [[True, False], [16, 32, 64, 128], [8, 16, 32]]
additional_layer, n_neurons, n_batch_size = config
possible_combinations = list(itertools.product(additional_layer, n_neurons, n_batch_size))

# Initialize lists to store results
validation_loss = np.zeros((5, len(possible_combinations)))

print('Possible combinations: ', len(possible_combinations))
print('\n')

In [ ]:
k = 5
kfold = KFold(n_splits=k, shuffle=True)
losses = np.empty(len(possible_combinations))

for i, (add_layer, neurons, batch_size) in enumerate(possible_combinations):
    print('--------------------------------------------------------------------')
    print(f'{i+1}th combination:')
    print('Additional Layer:', add_layer)
    print('Neurons:', neurons)
    print('Batch Size:', batch_size)
    #losses = np.empty(len(possible_combinations))
    val_loss = []
    train_loss = []
    
    for train_index, val_index in kfold.split(X_train_scaled):
        fold_train_X = X_train_scaled[train_index]
        fold_val_X = X_train_scaled[val_index]
        fold_train_y = y_train.iloc[train_index].values
        fold_val_y = y_train.iloc[val_index].values

        fold_train_X2 = fold_train_X.reshape((fold_train_X.shape[0], 1, fold_train_X.shape[1]))
        fold_val_X2 = fold_val_X.reshape((fold_val_X.shape[0], 1, fold_val_X.shape[1]))

        es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=20)

        model = Sequential()
        model.add(GRU(units=neurons, return_sequences=True, input_shape=(1, X_train.shape[1])))
        model.add(Dropout(0.2))

        if add_layer:
            model.add(GRU(units=neurons, return_sequences=True))
            model.add(Dropout(0.2))

        model.add(GRU(units=neurons, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(GRU(units=neurons, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(GRU(units=neurons, return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(units=1, activation='tanh'))
        model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

        model.fit(fold_train_X2, fold_train_y, epochs=100, verbose=0, batch_size=batch_size, callbacks=[es], shuffle=False)

        train_accuracy = model.evaluate(fold_train_X2, fold_train_y, verbose=0)
        val_accuracy = model.evaluate(fold_val_X2, fold_val_y, verbose=0)
        #print(f'train loss: {train_accuracy[0]}     val loss: {val_accuracy[0]}')

        train_loss.append(train_accuracy[0])
        val_loss.append(val_accuracy[0])

        
        print(f"Completed fold {len(train_loss)} with train loss: {train_accuracy[0]} and val loss: {val_accuracy[0]}")

    mean_train_loss = sum(train_loss) / len(train_loss)
    mean_val_loss = sum(val_loss) / len(val_loss)
    print(f'train loss: {mean_train_loss}     val loss: {mean_val_loss}')
    losses[i] = mean_val_loss

    # Clear the session to free up memory
    tf.keras.backend.clear_session()

# Identify the best hyperparameters based on validation loss
best_index = np.argmin(losses)
print(f"Best hyperparameters: {possible_combinations[best_index]} with validation loss: {losses[best_index]}")